In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
# pip install lxml

In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Jake\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [4]:
# URL of page to be scraped
url = 'https://redplanetscience.com/'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [5]:
# Retrieve the latest news title and paragraph
news_title=soup.find_all('div', class_='content_title')[0].text
news_p=soup.find_all('div', class_='article_teaser_body')[0].text
news_title
news_p

'Along with capturing an image before its steepest ascent ever, the robotic explorer filmed its "selfie stick," or robotic arm, in action.'

In [6]:
# URL of page to be scraped
photo_url = 'https://spaceimages-mars.com'
browser.visit(photo_url)
html2=browser.html

soup2=bs(html2,'html.parser')

In [7]:
mars=soup2.find_all("a", class_="fancybox-thumbs")[0]["href"]
featured_image_url = photo_url+"/"+mars
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [8]:
facts_url = 'https://galaxyfacts-mars.com'
browser.visit(facts_url)
html3=browser.html
soup3=bs(html3,'html.parser')

In [9]:
tables=pd.read_html(facts_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [10]:
mars_facts=tables[1]
mars_facts=mars_facts.rename(columns={0:"Measurement", 1:"Value"}).set_index("Measurement")
mars_facts

,Value
Measurement,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 ( Phobos & Deimos )
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
fact_table=mars_facts.to_html().replace("\n","")

In [12]:
print(fact_table)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Measurement</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>


In [13]:
#hemisphere info
hemis_url = 'https://marshemispheres.com/'
browser.visit(hemis_url)
html4=browser.html
soup4=bs(html4,'html.parser')

In [27]:
mars_links = soup4.find('div', class_='collapsible results')
mars_hemis = mars_links.find_all('div', class_='item')

hemisphere_image_urls=[]

for item in mars_hemis:
        # Extract title
        hem=item.find('div',class_='description')
        title=hem.h3.text
        # Extract image url
        hem_url=hem.a['href']
        browser.visit(hemis_url+hem_url)
        html=browser.html
        soup=bs(html,'html.parser')
        image_url=soup.find('li').a['href']
        image_src=(hemis_url+image_url)
        # Create dictionary for title and url
        hem_dict={
            'title':title,
            'image_url':image_src
        }
        hemisphere_image_urls.append(hem_dict)

In [28]:
hem_dict

{'title': 'Valles Marineris Hemisphere Enhanced',
 'image_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}